# Assignment 4

\- Yash Gupta (190997)

I am giving you a dataset that contains 70 judgments a subject has made about the size of hypothetical people based on their weight (in kilos) and height (in inches). The subject has categorized people into three categories - small, average and large. 

The dataset data.csv contains the 70 actual judgments made by the subject as a 70x3 matrix. The first column contains weights, the second contains heights. The third column contains the category label assigned by the subject (small = 1, average = 2, large = 3). 

I am also giving you a test set test.csv of 10 more weight-height combinations as a 10x2 matrix (same column interpretations). I want you to tell me what a generalized context model would predict this subjects' category labels to be, assuming  
(i) he is polite, and is far more likely to call someone big average than large  
(ii) He is more likely to use weight than height to make category judgments about size.

### Q1. (30 points)

Implement a GCM encoding these assumptions and give me quantitative predictions on the test set. Submit both code and category responses for the data points. 

In [6]:
# importing libraries
import numpy as np
import pandas as pd

Let's import the data

In [7]:
data = pd.read_csv('data.csv', names=['weight', 'height', 'category'])
data

,weight,height,category
0,48,58,1
1,54,62,1
2,48,56,1
3,46,62,1
4,47,59,1
...,...,...,...
65,78,67,3
66,76,68,3
67,79,64,3
68,82,63,3


We will redefine the category labels to be: small = 0, average = 1, large = 2. 

In [8]:
data.category -= 1
data

,weight,height,category
0,48,58,0
1,54,62,0
2,48,56,0
3,46,62,0
4,47,59,0
...,...,...,...
65,78,67,2
66,76,68,2
67,79,64,2
68,82,63,2


Let's define the functions required for GCM

In [9]:
def dist(x, y, alpha):
    dim = len(x)
    d = 0
    for i in range(dim):
        d += alpha[i] * (x[i] - y[i])
    d = abs(d)
    return d

def similarity(d, beta):
    s = np.exp(-beta * d)
    return s

def probability(data, y, r, alpha, beta, gamma):
    n = len(data)
    votes_r = 0
    for i in range(n):
        if data[i, -1] == r:
            d = dist(data[i, :-1], y, alpha)
            s = similarity(d, beta)
            votes_r += gamma[r] * s
    
    votes_total = 0
    for i in range(n):
        d = dist(data[i, :-1], y, alpha)
        s = similarity(d, beta)
        r = data[i, -1]
        votes_total += gamma[r] * s
    
    p_r = votes_r / votes_total
    return p_r

def gcm(data, y, num_cat, alpha, beta, gamma):
    p = np.empty(num_cat)
    for i in range(num_cat):
        p[i] = probability(data, y, i, alpha, beta, gamma)
    pred = np.argmax(p)
    return pred

Let's import the test data

In [10]:
y = pd.read_csv('test.csv', names=['weight', 'height'])
y

,weight,height
0,74,67
1,69,63
2,92,81
3,64,61
4,66,84
5,76,68
6,61,58
7,64,76
8,68,66
9,34,61


Now, let's classify the test data

In [11]:
n_test = len(y)
alpha = [2 / 3, 1 / 3]
beta = 1
gamma = [1, 2, 0.5]
preds = np.empty(n_test, dtype=int)
for i in range(n_test):
    preds[i] = gcm(np.array(data), np.array(y.iloc[i]), 3, alpha, beta, gamma)

for i in range(n_test):
    print(f"Weight: {y.loc[i, 'weight']}, Height: {y.loc[i, 'height']}, Category: {preds[i]}")

Weight: 74, Height: 67, Category: 1
Weight: 69, Height: 63, Category: 1
Weight: 92, Height: 81, Category: 2
Weight: 64, Height: 61, Category: 1
Weight: 66, Height: 84, Category: 1
Weight: 76, Height: 68, Category: 2
Weight: 61, Height: 58, Category: 1
Weight: 64, Height: 76, Category: 1
Weight: 68, Height: 66, Category: 1
Weight: 34, Height: 61, Category: 0


### Q2. (40 points)

I am also sharing with you, John McDonnell's python implementation of Anderson's Rational Model of Categorization (rational.py). Modify the code to obtain category predictions for the data I have shared with you. 

In [12]:
# Implementation of Anderson's venerable "rational" model of categorization.
# Assumes that stimuli were generated by a mixture of Gaussian distributions;
# rather than compute the full Bayesian posterior, it views items sequentially
# and assigns each to the maximum a posteriori cluster.
#
# At the end it is presented with a stimulus with one item missing, and
# predicts the probability that its value is a '0' or a '1'.
#
# Implemented in python by John McDonnell
#
# References: Anderson (1990) and Anderson (1991),

import numpy as np
from random import shuffle

#Utility functions:

class dLocalMAP:
    """
    See Anderson (1990, 1991)
    'Categories' renamed 'clusters' to avoid confusion.
    Discrete version.
    
    Stimulus format is a list of integers from 0 to n-1 where n is the number
    of possible features (e.g. [1,0,1])
    
    args: c, alphas
    """
    
    def __init__(self, args):
        self.partition = [[]]
        self.c, self.alpha = args
        self.alpha0 = sum(self.alpha.T)
        self.N = 0
    
    def probClustVal(self, k, i, val):
        """Find P(j|k)"""
        cj = len([x for x in self.partition[k] if x[i]==val])
        nk = len(self.partition)
        return (cj + self.alpha[i][val])/(nk + self.alpha0[i])
    
    def condclusterprob(self, stim, k):
        """Find P(F|k)"""
        pjks = []
        for i in range(len(stim)):
            cj = len([x for x in self.partition[k] if x[i]==stim[i]])
            nk = len(self.partition[k])
            pjks.append( (cj + self.alpha[i][stim[i]])/(nk + self.alpha0[i]) )
        return np.product( pjks )
        
    
    def posterior(self, stim):
        """Find P(k|F) for each cluster"""
        pk = np.zeros( len(self.partition) )
        pFk = np.zeros( len(self.partition) )
        
        # existing clusters:
        for k in range(len(self.partition)):
            pk[k] = self.c * len(self.partition[k])/ ((1-self.c) + self.c * self.N)
            if len(self.partition[k])==0: # case of new cluster
                pk[k] = (1-self.c) / (( 1-self.c ) + self.c * self.N)
            pFk[k] = self.condclusterprob( stim, k)
        
        # put it together
        pkF = (pk*pFk) # / sum( pk*pFk )
        
        return pkF
    
    def stimulate(self, stim):
        """Argmax of P(k|F) + P(0|F)"""
        winner = np.argmax( self.posterior(stim) )
        print("Stim: ", stim)
        print("Partition: ", self.partition)
        print(self.posterior(stim))
        
        if len(self.partition[winner]) == 0:
            self.partition.append( [] )
        self.partition[winner].append(stim)
        
        self.N += 1
    
    def query(self, stimulus):
        """Queried value should be -1."""
        qdim = -1
        for i in range(len(stimulus)):
            if stimulus[i] < 0:
                if qdim != -1:
                    raise Exception("ERROR: Multiple dimensions queried.")
                qdim = i
        
        self.N = sum([len(x) for x in self.partition])
        
        pkF = self.posterior(stimulus)
        pkF = pkF[:-1] / sum(pkF[:-1]) # eliminate `new cluster' prob
        
        pjF = np.array( [sum( [ pkF[k] * self.probClustVal(k, qdim, j) \
                for k in range(len(self.partition)-1)] ) 
                for j in range(len( self.alpha[qdim] ))] )
        
        return pjF / sum(pjF)

def testlocalmapD():
    """
    Tests the Anderson's ratinal model using the Medin & Schaffer (1978) data.
    
    This script will print out the probability that each item belongs to each
    of the existing clusters or to a new cluster, and the model assign it to
    the most likely cluster. To see that the model is working correctly, you
    can follow along with Anderson (1991), which steps through in the same way.
    """
    stims = [[1, 1, 1, 1, 1], # Medin & Schaffer (1978)
             [1, 0, 1, 0, 1], 
             [1, 0, 1, 1, 0],
             [0, 0, 0, 0, 0],
             [0, 1, 0, 1, 1],  
             [0, 1, 0, 0, 0]]
    # These are the classic Shepard Type II and Type IV datasets.
    # Uncomment the one you want to try out; you might want to uncomment
    # shuffling the stims too if you don't care about order.
    #stims = [[0, 0, 0, 0], [0, 0, 1, 0], [1, 1, 0, 1], [1, 1, 1, 1], [1, 0, 0, 0], [1, 0, 1, 1], [0, 1, 0, 0], [0, 1, 1, 1]] # Type IV
    stims = [[0, 0, 0, 0], [0, 0, 1, 0], [1, 1, 0, 0], [1, 1, 1, 0], [1, 0, 0, 1], [1, 0, 1, 1], [0, 1, 0, 1], [0, 1, 1, 1]] # Type II
    
    for _ in range(1):
        model = dLocalMAP([.5, np.ones((len(stims[0]),2))])
        
        shuffle(stims)
        for s in stims:
            model.stimulate(s)
        print(model.partition)
        
        print("Prob vals for 0,0,0,0,?", model.query( [0]*(len(stims[0])-1) + [-1] ))


In [40]:
# Implementation of Anderson's venerable "rational" model of categorization.
# Assumes that stimuli were generated by a mixture of Gaussian distributions;
# rather than compute the full Bayesian posterior, it views items sequentially
# and assigns each to the maximum a posteriori cluster.
#
# At the end it is presented with a stimulus with one item missing, and
# predicts the probability that its value is a '0' or a '1'.
#
# Implemented in python by John McDonnell
#
# References: Anderson (1990) and Anderson (1991),

import numpy as np
from random import shuffle

#Utility functions:

class dLocalMAP:
    """
    See Anderson (1990, 1991)
    'Categories' renamed 'clusters' to avoid confusion.
    Discrete version.
    
    Stimulus format is a list of integers from 0 to n-1 where n is the number
    of possible features (e.g. [1,0,1])
    
    args: c, alphas
    """
    
    def __init__(self, args):
        self.partition = [[]]
        self.c, self.alpha = args
        self.alpha0 = sum(self.alpha.T)
        self.N = 0
    
    def probClustVal(self, k, i, val):
        """Find P(j|k)"""
        cj = len([x for x in self.partition[k] if x[i]==val])
        nk = len(self.partition)
        return (cj + self.alpha[i][val])/(nk + self.alpha0[i])
    
    def condclusterprob(self, stim, k):
        """Find P(F|k)"""
        pjks = []
        for i in range(len(stim)):
            cj = len([x for x in self.partition[k] if x[i]==stim[i]])
            nk = len(self.partition[k])
            pjks.append( (cj + self.alpha[i][stim[i]])/(nk + self.alpha0[i]) )
        return np.product( pjks )
        
    
    def posterior(self, stim):
        """Find P(k|F) for each cluster"""
        pk = np.zeros( len(self.partition) )
        pFk = np.zeros( len(self.partition) )
        
        # existing clusters:
        for k in range(len(self.partition)):
            pk[k] = self.c * len(self.partition[k])/ ((1-self.c) + self.c * self.N)
            if len(self.partition[k])==0: # case of new cluster
                pk[k] = (1-self.c) / (( 1-self.c ) + self.c * self.N)
            pFk[k] = self.condclusterprob( stim, k)
        
        # put it together
        pkF = (pk*pFk) # / sum( pk*pFk )
        
        return pkF
    
    def stimulate(self, stim):
        """Argmax of P(k|F) + P(0|F)"""
        winner = np.argmax( self.posterior(stim) )
        print("Stim: ", stim)
        print("Partition: ", self.partition)
        print(self.posterior(stim))
        
        if len(self.partition[winner]) == 0:
            self.partition.append( [] )
        self.partition[winner].append(stim)
        
        self.N += 1
    
    def query(self, stimulus):
        """Queried value should be -1."""
        qdim = -1
        for i in range(len(stimulus)):
            if stimulus[i] < 0:
                if qdim != -1:
                    raise Exception("ERROR: Multiple dimensions queried.")
                qdim = i
        
        self.N = sum([len(x) for x in self.partition])
        
        pkF = self.posterior(stimulus)
        pkF = pkF[:-1] / sum(pkF[:-1]) # eliminate `new cluster' prob
        
        pjF = np.array( [sum( [ pkF[k] * self.probClustVal(k, qdim, j) \
                for k in range(len(self.partition)-1)] ) 
                for j in range(len( self.alpha[qdim] ))] )
        
        return pjF / sum(pjF)

In [29]:
def predict(p):
    pred = np.argmax(p)
    return pred

In [45]:
model = dLocalMAP([.5, np.ones((data.shape[1], 100))])
for s in np.array(data):
    model.stimulate(s)

Stim:  [48 58  0]
Partition:  [[]]
[1.e-06]
Stim:  [54 62  0]
Partition:  [[array([48, 58,  0])], []]
[9.70590148e-07 5.00000000e-07]
Stim:  [48 56  0]
Partition:  [[array([48, 58,  0]), array([54, 62,  0])], []]
[3.76928934e-06 3.33333333e-07]
Stim:  [46 62  0]
Partition:  [[array([48, 58,  0]), array([54, 62,  0]), array([48, 56,  0])], []]
[5.49084996e-06 2.50000000e-07]
Stim:  [47 59  0]
Partition:  [[array([48, 58,  0]), array([54, 62,  0]), array([48, 56,  0]), array([46, 62,  0])], []]
[3.55598543e-06 2.00000000e-07]
Stim:  [48 60  0]
Partition:  [[array([48, 58,  0]), array([54, 62,  0]), array([48, 56,  0]), array([46, 62,  0]), array([47, 59,  0])], []]
[1.29575640e-05 1.66666667e-07]
Stim:  [48 55  0]
Partition:  [[array([48, 58,  0]), array([54, 62,  0]), array([48, 56,  0]), array([46, 62,  0]), array([47, 59,  0]), array([48, 60,  0])], []]
[2.01508628e-05 1.42857143e-07]
Stim:  [51 57  0]
Partition:  [[array([48, 58,  0]), array([54, 62,  0]), array([48, 56,  0]), array(

In [46]:
n_test = len(y)
preds = np.empty(n_test, dtype=int)
for i in range(n_test):
    p = model.query(np.append(np.array(y.iloc[i]), -1))
    preds[i] = predict(p)

for i in range(n_test):
    print(f"Weight: {y.loc[i, 'weight']}, Height: {y.loc[i, 'height']}, Category: {preds[i]}")

Weight: 74, Height: 67, Category: 1
Weight: 69, Height: 63, Category: 1
Weight: 92, Height: 81, Category: 1
Weight: 64, Height: 61, Category: 1
Weight: 66, Height: 84, Category: 1
Weight: 76, Height: 68, Category: 1
Weight: 61, Height: 58, Category: 1
Weight: 64, Height: 76, Category: 1
Weight: 68, Height: 66, Category: 1
Weight: 34, Height: 61, Category: 1


In [28]:
testlocalmapD()

Stim:  [1, 1, 1, 1, 1]
Partition:  [[]]
[0.03125]
Stim:  [1, 0, 1, 0, 1]
Partition:  [[[1, 1, 1, 1, 1]], []]
[0.01646091 0.015625  ]
Stim:  [1, 0, 1, 1, 0]
Partition:  [[[1, 1, 1, 1, 1], [1, 0, 1, 0, 1]], []]
[0.0234375  0.01041667]
Stim:  [0, 0, 0, 0, 0]
Partition:  [[[1, 1, 1, 1, 1], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]], []]
[0.00288   0.0078125]
Stim:  [0, 1, 0, 1, 1]
Partition:  [[[1, 1, 1, 1, 1], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]], [[0, 0, 0, 0, 0]], []]
[0.003456   0.00329218 0.00625   ]
Stim:  [0, 1, 0, 0, 0]
Partition:  [[[1, 1, 1, 1, 1], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]], [[0, 0, 0, 0, 0]], [[0, 1, 0, 1, 1]], []]
[0.00128    0.01097394 0.00548697 0.00520833]
[[[1, 1, 1, 1, 1], [1, 0, 1, 0, 1], [1, 0, 1, 1, 0]], [[0, 0, 0, 0, 0], [0, 1, 0, 0, 0]], [[0, 1, 0, 1, 1]], []]
Prob vals for 0,0,0,0,? [0.68869481 0.31130519]


### Q3. (30 points)

For both GCM and RMC, show empirically using the dataset I've shared that both models assume exchangeability of data, viz. the order in which data enters the model does not affect the category labels of the model for any given subset of data. 